Что нужно сделать

К вам поступили разрозненные файлы с данными о действиях людей на каждом этапе воронки по нескольким рекламным кампаниям. В каждом файле хранится информация о каждом отдельном канале коммуникации. Вы можете опознать эти файлы по названиям рекламных кампаний.

Также в отдельном файле есть информация о затратах на рекламные кампании.

Вам нужно считать файлы средствами Python и собрать данные по специфике действий людей на всей воронке в единую Excel-таблицу. В таблице будут храниться данные по всем рекламным кампаниям со следующей структурой:

user_id: зашифрованный идентификатор пользователя;
ad_channel: канал коммуникации (например, поиск, видеореклама, баннерная реклама, партнёрские сети, органический трафик сайта);
funnel_stage: этап воронки;
price: затраты на рекламу;
profit: прибыль на этапе покупки, в остальных случаях маркируется как None;
os: операционная система, которой пользуется человек;
timestamp: время совершения действия юзером.
В колонку ad_channel добавьте название канала коммуникации, взяв его из соответствующего названия файла.

Итоговую таблицу отсортируйте по колонке timestamp. Эта таблица понадобится вам для работы в следующих модулях.



In [1]:
import pandas as pd
import numpy as np
import json

In [37]:
df_banner = pd.read_excel('Копия Копия banner.xlsx')
df_banner['ad_channel'] = 'banner'
df_banner

,user_id,funnel_stage,timestamp,profit,os,ad_channel
0,666743,interest,150000,NaN,Android,banner
1,47061,interest,150000,NaN,Android,banner
2,587720,interest,150000,NaN,Android,banner
3,991300,intent,150001,NaN,iOS,banner
4,289020,interest,150001,NaN,Android,banner
...,...,...,...,...,...,...
730445,319234,intent,201458,NaN,Android,banner
730446,113762,intent,201478,NaN,Android,banner
730447,874686,purchase,201579,520.0,iOS,banner
730448,574291,intent,201581,NaN,Android,banner


In [36]:
df_rich = pd.read_excel('Копия Копия rich.xlsx')
df_rich['ad_channel'] = 'rich'
df_rich

,user_id,funnel_stage,timestamp,profit,os,ad_channel
0,278945,INTEREST,150000,NaN,iOS,rich
1,246655,INTEREST,150004,NaN,iOS,rich
2,777208,INTEREST,150009,NaN,Android,rich
3,737227,INTEREST,150009,NaN,iOS,rich
4,599994,INTEREST,150015,NaN,Android,rich
...,...,...,...,...,...,...
73619,28732,INTENT,201143,NaN,iOS,rich
73620,817066,INTENT,201255,NaN,iOS,rich
73621,827850,INTENT,201446,NaN,iOS,rich
73622,732690,INTENT,201462,NaN,iOS,rich


In [32]:
df_advertising_information = pd.read_json('Копия Копия advertising_information.json')
#df_advertising_information.price.fillna(0, inplace=True)
df_advertising_information

,ad_channel,price
0,search,1.50
1,banner,0.50
2,video,0.75
3,native,2.00
4,partner_network,1.00
5,rich,2.50
6,organic_traffic,0.00
7,other,NaN


In [35]:
df_native = pd.read_csv('Копия Копия native.csv', index_col  = False)
df_native = df_native.drop(columns=['Unnamed: 0'])
df_native['ad_channel'] = 'native'
df_native

,user_id,funnel_stage,timestamp,profit,os,ad_channel
0,602424,interest,150002,NaN,iOS,native
1,217793,interest,150010,NaN,iOS,native
2,978521,interest,150010,NaN,iOS,native
3,747938,interest,150013,NaN,iOS,native
4,809853,interest,150016,NaN,iOS,native
...,...,...,...,...,...,...
130596,704762,consideration,200781,NaN,iOS,native
130597,659956,consideration,200792,NaN,iOS,native
130598,500938,consideration,200843,NaN,iOS,native
130599,388053,purchase,201095,2136.0,Android,native


In [49]:
df_organic_traffic = pd.read_csv('Копия Копия organic_traffic.csv', header=None)
df_organic_traffic = df_organic_traffic.drop(columns=[0])
df_organic_traffic['ad_channel'] = 'organic_traffic'
df_organic_traffic.columns = ['user_id', 'funnel_stage', 'timestamp', 'profit', 'os', 'ad_channel']
df_organic_traffic

,user_id,funnel_stage,timestamp,profit,os,ad_channel
0,764740,interest,150001,NaN,iOS,organic_traffic
1,476137,interest,150002,NaN,Android,organic_traffic
2,959652,interest,150002,NaN,Android,organic_traffic
3,227224,interest,150004,NaN,Android,organic_traffic
4,541742,interest,150006,NaN,Android,organic_traffic
...,...,...,...,...,...,...
607354,946024,intent,201892,NaN,Android,organic_traffic
607355,1036500,purchase,202012,50.0,Android,organic_traffic
607356,539693,purchase,202053,50.0,iOS,organic_traffic
607357,583326,purchase,202140,1099.0,Android,organic_traffic


In [39]:
df_other = pd.read_csv('Копия Копия other.csv')
df_other = df_other.drop(columns=['Unnamed: 0'])
df_other['ad_channel'] = 'other'
df_other

,user_id,funnel_stage,timestamp,profit,os,ad_channel
0,866112,interest,150009,NaN,Android,other
1,634356,interest,150014,NaN,iOS,other
2,160340,interest,150030,NaN,Android,other
3,1004246,interest,150036,NaN,Android,other
4,897963,interest,150048,NaN,Android,other
...,...,...,...,...,...,...
21326,109868,intent,200943,NaN,Android,other
21327,820384,intent,201041,NaN,Android,other
21328,394552,intent,201073,NaN,Android,other
21329,394552,purchase,201184,50.0,Android,other


In [50]:
df_partner_network = pd.read_csv('Копия Копия partner_network.csv', header=None)
df_partner_network = df_partner_network.drop(columns=[0])
df_partner_network['ad_channel'] = 'partner_network'
df_partner_network.columns = ['user_id', 'funnel_stage', 'timestamp', 'profit', 'os', 'ad_channel']
df_partner_network

,user_id,funnel_stage,timestamp,profit,os,ad_channel
0,561010,interest,150000,NaN,iOS,partner_network
1,800792,interest,150000,NaN,Android,partner_network
2,1035593,interest,150000,NaN,Android,partner_network
3,863728,interest,150002,NaN,iOS,partner_network
4,593703,interest,150002,NaN,Android,partner_network
...,...,...,...,...,...,...
461867,548770,intent,201348,NaN,iOS,partner_network
461868,734166,purchase,201375,520.0,Android,partner_network
461869,214696,intent,201381,NaN,iOS,partner_network
461870,909107,intent,201867,NaN,iOS,partner_network


In [42]:
df_video = pd.read_csv('Копия Копия video.csv')
df_video = df_video.drop(columns=['Unnamed: 0'])
df_video['ad_channel'] = 'video'
df_video

,user_id,funnel_stage,timestamp,profit,os,ad_channel
0,512217,interest,150000,NaN,iOS,video
1,833649,interest,150000,NaN,Android,video
2,327813,interest,150002,NaN,Android,video
3,241345,interest,150004,NaN,iOS,video
4,212554,interest,150004,NaN,Android,video
...,...,...,...,...,...,...
712334,446310,consideration,200926,NaN,Android,video
712335,340340,consideration,200927,NaN,Android,video
712336,313410,consideration,200951,NaN,iOS,video
712337,32380,consideration,200953,NaN,Android,video


In [51]:
data_frames = [df_banner, df_rich, df_native, df_organic_traffic, df_other, df_partner_network, df_video]  

result_df = pd.concat(data_frames, ignore_index=True)

result_df

,user_id,funnel_stage,timestamp,profit,os,ad_channel
0,666743,interest,150000,NaN,Android,banner
1,47061,interest,150000,NaN,Android,banner
2,587720,interest,150000,NaN,Android,banner
3,991300,intent,150001,NaN,iOS,banner
4,289020,interest,150001,NaN,Android,banner
...,...,...,...,...,...,...
2737571,446310,consideration,200926,NaN,Android,video
2737572,340340,consideration,200927,NaN,Android,video
2737573,313410,consideration,200951,NaN,iOS,video
2737574,32380,consideration,200953,NaN,Android,video


In [53]:
conditions = [
    (result_df['ad_channel'] == 'search'),
    (result_df['ad_channel'] == 'banner'),
    (result_df['ad_channel'] == 'video'),
    (result_df['ad_channel'] == 'native'),
    (result_df['ad_channel'] == 'partner_network'),
    (result_df['ad_channel'] == 'rich'),
    (result_df['ad_channel'] == 'organic_traffic')
]

# Задаем значения в соответствии с условиями
values = [1.50, 0.50, 0.75, 2.00, 1.00, 2.50, 0.00]

# Создаем новую колонку 'price' в зависимости от канала рекламы
result_df['price'] = np.select(conditions, values, default=np.nan) 
result_df

,user_id,funnel_stage,timestamp,profit,os,ad_channel,price
0,666743,interest,150000,NaN,Android,banner,0.50
1,47061,interest,150000,NaN,Android,banner,0.50
2,587720,interest,150000,NaN,Android,banner,0.50
3,991300,intent,150001,NaN,iOS,banner,0.50
4,289020,interest,150001,NaN,Android,banner,0.50
...,...,...,...,...,...,...,...
2737571,446310,consideration,200926,NaN,Android,video,0.75
2737572,340340,consideration,200927,NaN,Android,video,0.75
2737573,313410,consideration,200951,NaN,iOS,video,0.75
2737574,32380,consideration,200953,NaN,Android,video,0.75


# 7.6

In [34]:
america_personal = pd.read_csv("Employee_personal_Data_america.csv")
america_personal.head(5)

,Employee ID,Full Name,Gender,Ethnicity,Age,Country,City
0,E02002,Kai Le,Male,Asian,47,United States,Columbus
1,E02003,Robert Patel,Male,Asian,58,United States,Chicago
2,E02005,Harper Castillo,Female,Latino,39,United States,Seattle
3,E02006,Harper Dominguez,Female,Latino,42,United States,Austin
4,E02007,Ezra Vu,Male,Asian,62,United States,Phoenix


In [35]:
china_personal = pd.read_csv("Employee_personal_Data_china.csv")
china_personal.head(5)

,Employee ID,Full Name,Gender,Ethnicity,Age,Country,City
0,E02004,Cameron Lo,Male,Asian,34,China,Shanghai
1,E02008,Jade Hu,Female,Asian,58,China,Chongqing
2,E02009,Miles Chang,Male,Asian,62,China,Chengdu
3,E02014,Jose Wong,Male,Asian,45,China,Chongqing
4,E02018,Bella Tran,Female,Asian,45,China,Chengdu


In [41]:
personal_data = pd.concat([america_personal, china_personal], axis=0)
personal_data.head()

,Employee ID,Full Name,Gender,Ethnicity,Age,Country,City
0,E02002,Kai Le,Male,Asian,47,United States,Columbus
1,E02003,Robert Patel,Male,Asian,58,United States,Chicago
2,E02005,Harper Castillo,Female,Latino,39,United States,Seattle
3,E02006,Harper Dominguez,Female,Latino,42,United States,Austin
4,E02007,Ezra Vu,Male,Asian,62,United States,Phoenix


In [37]:
print(america_personal.shape)
print(china_personal.shape)
print(personal_data.shape)

(773, 7)
(227, 7)
(1000, 7)


In [40]:
pd.concat([america_personal, china_personal], axis=0, keys=["america", "asia"]).head()

Employee ID         Full Name  Gender Ethnicity  Age        Country  \
america 0      E02002            Kai Le    Male     Asian   47  United States   
        1      E02003      Robert Patel    Male     Asian   58  United States   
        2      E02005   Harper Castillo  Female    Latino   39  United States   
        3      E02006  Harper Dominguez  Female    Latino   42  United States   
        4      E02007           Ezra Vu    Male     Asian   62  United States   

               City  
america 0  Columbus  
        1   Chicago  
        2   Seattle  
        3    Austin  
        4   Phoenix

In [43]:
job_data = pd.read_excel("Employee_job_Data.xlsx")
job_data.head()

,Employee ID,Job Title,Department,Business Unit,Hire Date,Annual Salary,Bonus %,Exit Date
0,E02002,Controls Engineer,Engineering,Manufacturing,2/5/2022,"$92,368",0%,NaN
1,E02003,Analyst,Sales,Corporate,10/23/2013,"$45,703",0%,NaN
2,E02004,Network Administrator,IT,Research & Development,3/24/2019,"$83,576",0%,NaN
3,E02005,IT Systems Architect,IT,Corporate,4/7/2018,"$98,062",0%,NaN
4,E02006,Director,Engineering,Corporate,6/18/2005,"$175,391",24%,NaN


In [45]:
all_data = pd.merge(personal_data, job_data,
                    how='inner', on='Employee ID')
all_data.head()

,Employee ID,Full Name,Gender,Ethnicity,Age,Country,City,Job Title,Department,Business Unit,Hire Date,Annual Salary,Bonus %,Exit Date
0,E02002,Kai Le,Male,Asian,47,United States,Columbus,Controls Engineer,Engineering,Manufacturing,2/5/2022,"$92,368",0%,NaN
1,E02003,Robert Patel,Male,Asian,58,United States,Chicago,Analyst,Sales,Corporate,10/23/2013,"$45,703",0%,NaN
2,E02005,Harper Castillo,Female,Latino,39,United States,Seattle,IT Systems Architect,IT,Corporate,4/7/2018,"$98,062",0%,NaN
3,E02006,Harper Dominguez,Female,Latino,42,United States,Austin,Director,Engineering,Corporate,6/18/2005,"$175,391",24%,NaN
4,E02007,Ezra Vu,Male,Asian,62,United States,Phoenix,Network Administrator,IT,Manufacturing,4/22/2004,"$66,227",0%,2/14/2014


In [26]:
df1 = np.genfromtxt("MRI2_1.csv", delimiter=',', skip_header=1)
df2 = np.genfromtxt("MRI2_2.csv", delimiter=',', skip_header=1)
df3 = np.genfromtxt("MRI2_3.csv", delimiter=',', skip_header=1)

In [27]:
np.hstack([np.vstack([df1, df2]), df3]).shape

(735, 9)

In [32]:
a = np.array((1,2,3))
b = np.array((2,3,4))
print(a.shape)
c = np.dstack((a,b))
print(c, c.shape)

(3,)
[[[1 2]
  [2 3]
  [3 4]]] (1, 3, 2)


In [83]:
column1 = df1[:, 0]
column2 = df1[:, 1]

In [86]:
np.column_stack([column1, column2]).shape

(399, 2)

In [87]:
np.row_stack([column1, column2]).shape

(2, 399)

In [88]:
np.concatenate([np.concatenate([df1, df2], axis=0), df3], axis=1).shape

(735, 9)